In [5]:
import requests
from requests import Response
from pydantic import root_model,BaseModel
from pprint import pprint
from pydantic import BaseModel, RootModel, Field,field_validator
from pprint import pprint
import pandas as pd

url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
class Data(BaseModel):
    #擷取資料
    where:str=Field(alias="sna")
    nmae:str=Field(alias="sarea")
    total:int=Field(alias="tot")
    boIn:int=Field(alias="sbi")
    boOut:int=Field(alias="bemp")
    data:str=Field(alias="mday")
    field_validator("total","boIn","boOut",mode='before')
    #避免特定資列為空字串
    @classmethod
    def empty_to_zero(cls,value):
        if value=='':
            return '0'
        else:
            return value  


class AllData(RootModel):
    root:list[Data]
    def __iter__(self) :
        return iter(self.root)
    def __getitem__(self,ind):
        return self.root[ind]
    
    
def get_request(link:str)->Response|None:
    try:
        re:Response=requests.get(link)
        re.raise_for_status()
        if re.status_code==200:
            print("Asset")
            return re
        else:
            print("Fail")
    except:
        print("Fail")
        return None

def main()->int:
    req:requests=get_request(url)
    if req==None:
        return -1
    ind=req.text
    data:Data=AllData.model_validate_json(ind)
    dataDic=data.model_dump()
    display(pd.DataFrame(dataDic))
    
    return 0
print(main())





Asset


,where,nmae,total,boIn,boOut,data
0,YouBike2.0_捷運科技大樓站,大安區,28,5,23,2024-04-14 10:33:20
1,YouBike2.0_復興南路二段273號前,大安區,21,14,7,2024-04-14 10:33:20
2,YouBike2.0_國北教大實小東側門,大安區,16,1,15,2024-04-14 10:35:19
3,YouBike2.0_和平公園東側,大安區,11,0,11,2024-04-14 10:27:13
4,YouBike2.0_辛亥復興路口西北側,大安區,16,6,10,2024-04-14 10:29:18
...,...,...,...,...,...,...
1401,YouBike2.0_臺大總圖書館西南側,臺大公館校區,30,20,10,2024-04-14 10:32:19
1402,YouBike2.0_臺大黑森林西側,臺大公館校區,20,8,12,2024-04-14 10:28:18
1403,YouBike2.0_臺大獸醫館南側,臺大公館校區,24,9,15,2024-04-14 10:24:18
1404,YouBike2.0_臺大新體育館東南側,臺大公館校區,40,33,2,2024-04-14 10:26:19


0


In [3]:
import requests
from requests import Response
import pandas as pd

from pydantic import BaseModel, RootModel, Field

class BikeOfSite(BaseModel):
    行政區:str = Field(alias='sarea')
    站點名稱:str = Field(alias='sna')
    時間:str = Field(alias='mday')
    總車輛數:int = Field(alias='tot')
    可借:int = Field(alias='sbi')
    可還:int = Field(alias='bemp')

############################

class BikeOfSites(RootModel):
    root:list[BikeOfSite]

    def __iter__(self):
        return iter(self.root)
    
    def __getitem__(self, idx):
        return self.root[idx]

############################

def get_response() -> Response | None:
    url_path:str = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'

    print(f'開始下載:')

    try:
        response:Response = requests.get(url_path, timeout=15)
        response.raise_for_status()
    except Exception as e:
        print('連線失敗。')
        print(e)
        return None
    
    if response.status_code != 200:
        print('下載失敗。')
        print(response.reason)
        return None
    
    print('下載成功')
    return response

############################

def jsonToList(raw:str) -> list[dict] | None:
    try:
        datas:BikeOfSites = BikeOfSites.model_validate_json(raw)
        items:list[dict] = datas.model_dump()
        return items
    except Exception as e:
        print('預想外的格式。')
        print(e)
        return None

############################

def main():
    response:Response | None = get_response()

    if not response:
        return
    
    rawString:str = response.text
    download_data:list[dict] | None = jsonToList(rawString)

    if not download_data:
        return

    df = pd.DataFrame(download_data)

    display(df)
    
main()

開始下載:
下載成功


,行政區,站點名稱,時間,總車輛數,可借,可還
0,大安區,YouBike2.0_捷運科技大樓站,2024-04-14 10:01:22,28,5,23
1,大安區,YouBike2.0_復興南路二段273號前,2024-04-14 09:59:20,21,5,16
2,大安區,YouBike2.0_國北教大實小東側門,2024-04-14 09:56:20,16,1,15
3,大安區,YouBike2.0_和平公園東側,2024-04-14 09:56:20,11,3,8
4,大安區,YouBike2.0_辛亥復興路口西北側,2024-04-14 09:59:19,16,8,8
...,...,...,...,...,...,...
1401,臺大公館校區,YouBike2.0_臺大總圖書館西南側,2024-04-14 09:58:19,30,9,19
1402,臺大公館校區,YouBike2.0_臺大黑森林西側,2024-04-14 09:58:19,20,6,14
1403,臺大公館校區,YouBike2.0_臺大獸醫館南側,2024-04-14 09:54:19,24,7,17
1404,臺大公館校區,YouBike2.0_臺大新體育館東南側,2024-04-14 09:59:20,40,30,5
